In [6]:
from pathlib import Path
import pandas as pd

from make_summary import summarize_group
from mil.utils import human_format

selected_model_type = "mnist_collage"
selected_model_type = "mnist_collage_ablations"
selected_model_type = "camelyon16"

if "mnist_collage" in selected_model_type:
    datasets = ["mnist_collage", "mnist_collage_inverse"]
else:
    datasets = ["camelyon16"]
yaml_folder = Path("conf") / "selected_model" / selected_model_type

In [7]:
models = [file.stem for file in yaml_folder.glob("*.yaml") if file.name[0] != "_"]
if selected_model_type == "mnist_collage_ablations":
    models.append("distance_aware_self_attention")
models.sort()
models

['abmil',
 'discrete_rel_pos_self_attention',
 'distance_aware_self_attention',
 'distance_aware_self_attention_fc',
 'distance_aware_self_attention_fc_t3',
 'just_pool',
 'self_attention',
 'self_attention_axial_pe',
 'self_attention_fourier_pe',
 'transmil',
 'transmil_ourparams']

In [8]:
all_stats = {
    dataset: {
        model: summarize_group(f"selected-{dataset}-{model}", log_to_wandb=False)
        for model in models
    } for dataset in datasets
}
dfs = []
for dataset, dataset_stats in all_stats.items():
    df = pd.DataFrame(dataset_stats).T
    df["dataset"] = dataset
    dfs.append(df)

# Merge
df = pd.concat(dfs)
df.index.name = "model"
df.reset_index(inplace=True)
df.set_index(["dataset", "model"], inplace=True)
df[["mean(train/acc)", "std(train/acc)", "mean(test/acc)", "std(test/acc)", "mean(train/auc)", "std(train/auc)", "mean(test/auc)", "std(test/auc)"]].round(3)

2023-05-09 00:13:13.288 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-abmil
2023-05-09 00:13:15.478 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-discrete_rel_pos_self_attention
2023-05-09 00:13:17.874 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-distance_aware_self_attention
2023-05-09 00:13:20.416 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-distance_aware_self_attention_fc
2023-05-09 00:13:22.900 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-distance_aware_self_attention_fc_t3
2023-05-09 00:13:24.968 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-just_pool
2023-05-09 00:13:27.012 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-camelyon16-self_attention
2023-05-09 00:13:29.301 | INFO     | make_summary:summarize_group:43 - Summ

mean(train/acc)  \
dataset    model                                                  
camelyon16 abmil                                          0.928   
           discrete_rel_pos_self_attention                0.931   
           distance_aware_self_attention                  0.923   
           distance_aware_self_attention_fc               0.978   
           distance_aware_self_attention_fc_t3            0.993   
           just_pool                                      0.630   
           self_attention                                 0.930   
           self_attention_axial_pe                        0.883   
           self_attention_fourier_pe                      0.935   
           transmil                                       0.997   
           transmil_ourparams                             0.961   

                                                std(train/acc)  \
dataset    model                                                 
camelyon16 abmil                                         0.004   
           discrete_rel_pos_self_attention               0.016   
           distance_aware_self_attention                 0.015   
           distance_aware_self_attention_fc              0.021   
           distance_aware_self_attention_fc_t3           0.008   
           just_pool                                     0.039   
           self_attention                                0.014   
           self_attention_axial_pe                       0.019   
           self_attention_fourier_pe                     0.011   
           transmil                                      0.007   
           transmil_ourparams                            0.019   

                                                mean(test/acc)  std(test/acc)  \
dataset    model                                                                
camelyon16 abmil                                         0.809          0.018   
           discrete_rel_pos_self_attention               0.825          0.010   
           distance_aware_self_attention                 0.831          0.011   
           distance_aware_self_attention_fc              0.887          0.022   
           distance_aware_self_attention_fc_t3           0.878          0.020   
           just_pool                                     0.648          0.069   
           self_attention                                0.829          0.023   
           self_attention_axial_pe                       0.521          0.031   
           self_attention_fourier_pe                     0.791          0.067   
           transmil                                      0.884          0.026   
           transmil_ourparams                            0.879          0.020   

                                                mean(train/auc)  \
dataset    model                                                  
camelyon16 abmil                                          0.964   
           discrete_rel_pos_self_attention                0.976   
           distance_aware_self_attention                  0.974   
           distance_aware_self_attention_fc               0.997   
           distance_aware_self_attention_fc_t3            0.999   
           just_pool                                      0.686   
           self_attention                                 0.972   
           self_attention_axial_pe                        0.956   
           self_attention_fourier_pe                      0.975   
           transmil                                       0.999   
           transmil_ourparams                             0.991   

                                                std(train/auc)  \
dataset    model                                                 
camelyon16 abmil                                         0.003   
           discrete_rel_pos_self_attention               0.006   
           distance_aware_self_attention                 0.004   
           distance_aware_self_attention_fc              0.00

In [12]:
if selected_model_type == "mnist_collage":
    NAMES = {
        "just_pool": "MIL with max pool",
        "abmil": "AB-MIL~\\cite{ilse2018attention}",
        "gnn_gat": "MIL with GNN (GAT~\\cite{velickovic2018graph})",
        "gnn_gcn": "MIL with GNN (GCN~\\cite{kipf2017semisupervised})",
        # "mil_gnn": "MIL-GNN~\\cite{tu2019multiple}",
        # "mil_gnn_ds": "MIL-GNN-DS~\\cite{tu2019multiple}",
        "mil_gnn_ds": "MIL-GNN~\\cite{tu2019multiple}",
        "induced_set_transformer": "MIL with iSet Transformer~\\cite{lee2019set}",
        "set_transformer": "MIL with Set Transformer~\\cite{lee2019set}",
        "self_attention": "MIL with SA~\\cite{vaswani2017attention}",
        "self_attention_axial_pe": "MIL with SA + axial PE~\\cite{ramachandran2019stand}",
        "self_attention_fourier_pe": "MIL with SA + Fourier PE~\\cite{yang2021learnable}",
        "discrete_rel_pos_self_attention": "MIL with disc.\\ rel.\\ SA~\\cite{wu2021rethinking}",
        "transmil": "TransMIL~\\cite{shao2021transmil}",
        "distance_aware_self_attention": "DAS-MIL (ours)",
    }
elif selected_model_type == "mnist_collage_ablations":
    NAMES = {
        "distance_aware_self_attention_embedk": "DAS-MIL ($\\vb^K$)",
        "distance_aware_self_attention_embedq": "DAS-MIL ($\\vb^Q$)",
        "distance_aware_self_attention_embedv": "DAS-MIL ($\\vb^V$)",
        "distance_aware_self_attention_embedkq": "DAS-MIL ($\\vb^K, \\vb^Q$) ($+ \\vb^Q{\\vb^K}^\\top$ in \\cref{eq:das:compatibility:impl})",
        "distance_aware_self_attention_embedkq_noterm3": "DAS-MIL ($\\vb^K, \\vb^Q$)",
        "distance_aware_self_attention_embedkv": "DAS-MIL ($\\vb^K, \\vb^V$)",
        "distance_aware_self_attention_embedqv": "DAS-MIL ($\\vb^Q, \\vb^V$)",
        "distance_aware_self_attention_embedkqv": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$) ($+ \\vb^Q{\\vb^K}^\\top$ in \\cref{eq:das:compatibility:impl})",
        "line1": None,
        "distance_aware_self_attention_fixedembed": "DAS-MIL (non-trainable $\\vb^K, \\vb^Q, \\vb^V$)",
        "line2": None,
        # "distance_aware_self_attention_embedkqv_noterm3": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$)",
        "distance_aware_self_attention": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$)",
    }
elif selected_model_type == "camelyon16":
    NAMES = {
        "just_pool": "MIL with max pool",
        "abmil": "AB-MIL~\\cite{ilse2018attention}",
        "self_attention": "MIL with SA~\\cite{vaswani2017attention}",
        "self_attention_axial_pe": "MIL with SA + axial PE~\\cite{ramachandran2019stand}",
        "self_attention_fourier_pe": "MIL with SA + Fourier PE~\\cite{yang2021learnable}",
        'discrete_rel_pos_self_attention': "MIL with disc.\\ rel.\\ SA~\\cite{wu2021rethinking}",
        'transmil': "TransMIL~\\cite{shao2021transmil}",
        # 'transmil_ourparams': "TransMIL~\\cite{shao2021transmil}",
        # 'distance_aware_self_attention': "DAS-MIL (no FC)",
        'distance_aware_self_attention_fc': "DAS-MIL (ours)",
        # 'distance_aware_self_attention_fc_t3': "DAS-MIL (FC, T3)",
    }

pos_col = selected_model_type == "mnist_collage" or selected_model_type == "camelyon16"
def if_pos(s):
    return s if pos_col else ""

abs_pos = ["self_attention_axial_pe", "self_attention_fourier_pe", "transmil"]
rel_pos = ["discrete_rel_pos_self_attention", "distance_aware_self_attention_fc", "distance_aware_self_attention", "mil_gnn", "mil_gnn_ds", "gnn_gat", "gnn_gcn"]


if "mnist_collage" in selected_model_type:
    metrics = ["balanced_acc"]
    # metrics = ["auc"]
    print("\\begin{tabular}{l" + if_pos("|c") + "|r|rr|rr}")
    print("\\toprule")
    print(" & " + if_pos("&") + " & \\multicolumn{2}{c|}{\\smaller{MNIST-COLLAGE}} & \\multicolumn{2}{c}{\\smaller{MNIST-COLLAGE-INV}} \\\\")
    print("Model " + if_pos("& \\multicolumn{1}{c|}{Pos}") + " & \\multicolumn{1}{c|}{Params} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c|}{Test} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c}{Test} \\\\")
    print("\\midrule")

elif selected_model_type == "camelyon16":
    metrics = ["auc", "balanced_acc"]
    print("\\begin{tabular}{l|c|c|rr|rr}")
    print("\\toprule")
    print(" & & & \\multicolumn{2}{c|}{AUROC} & \\multicolumn{2}{c}{Balanced accuracy} \\\\")
    print("Model & Pos & \\multicolumn{1}{c|}{Params} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c|}{Test} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c}{Test} \\\\")
    print("\\midrule")

rows = {name: row for name, row in df.iterrows()}
for name, desc in NAMES.items():
    if desc is None:
        print("\\hline")
        continue
    pos = "abs" if name in abs_pos else "rel" if name in rel_pos else None
    row = f"{NAMES[name]}"
    pos = f" \\smaller{{{pos.upper()}}}" if pos in ("abs", "rel") else "\\xmark"
    row += if_pos(f" & {pos}")
    row += f" & {human_format(df.loc[datasets[0], name]['num_parameters'])}"


    for dataset in datasets:
        for metric in metrics:
            df_ds = df.loc[dataset]
            for split in ("train", "test"):
                if df_ds[f"mean({split}/{metric})"].loc[list(NAMES)].max().round(3) == df_ds[f"mean({split}/{metric})"].loc[name].round(3):
                    row += f" & \\textbf{{{df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}}}"
                else:
                    row += f" & {df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}"
    print(row + " \\\\")
print("\\bottomrule")
print("\\end{tabular}")

\begin{tabular}{l|c|c|rr|rr}
\toprule
 & & & \multicolumn{2}{c|}{AUROC} & \multicolumn{2}{c}{Balanced accuracy} \\
Model & Pos & \multicolumn{1}{c|}{Params} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c|}{Test} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c}{Test} \\
\midrule
MIL with max pool & \xmark & 769 & 0.686 $\pm$ 0.018 & 0.739 $\pm$ 0.017 & 0.639 $\pm$ 0.033 & 0.573 $\pm$ 0.077 \\
AB-MIL~\cite{ilse2018attention} & \xmark & 8.47K & 0.964 $\pm$ 0.003 & 0.795 $\pm$ 0.009 & 0.923 $\pm$ 0.004 & 0.773 $\pm$ 0.010 \\
MIL with SA~\cite{vaswani2017attention} & \xmark & 27.7K & 0.972 $\pm$ 0.014 & 0.823 $\pm$ 0.050 & 0.925 $\pm$ 0.016 & 0.803 $\pm$ 0.033 \\
MIL with SA + axial PE~\cite{ramachandran2019stand} &  \smaller{ABS} & 27.7K & 0.956 $\pm$ 0.016 & 0.485 $\pm$ 0.018 & 0.887 $\pm$ 0.017 & 0.490 $\pm$ 0.034 \\
MIL with SA + Fourier PE~\cite{yang2021learnable} &  \smaller{ABS} & 41K & 0.975 $\pm$ 0.007 & 0.819 $\pm$ 0.028 & 0.932 $\pm$ 0.011 & 0.770 $\pm$ 0.041 \\
MIL with disc.\ 

In [10]:
df

mean(train/acc)  \
dataset    model                                                  
camelyon16 abmil                                       0.927881   
           discrete_rel_pos_self_attention             0.930855   
           distance_aware_self_attention               0.922677   
           distance_aware_self_attention_fc            0.978439   
           distance_aware_self_attention_fc_t3         0.993309   
           just_pool                                   0.629740   
           self_attention                              0.930112   
           self_attention_axial_pe                     0.883271   
           self_attention_fourier_pe                   0.935316   
           transmil                                    0.997026   
           transmil_ourparams                          0.960595   

                                                std(train/acc)  \
dataset    model                                                 
camelyon16 abmil                                      0.004239   
           discrete_rel_pos_self_attention            0.016332   
           distance_aware_self_attention              0.015417   
           distance_aware_self_attention_fc           0.020930   
           distance_aware_self_attention_fc_t3        0.007619   
           just_pool                                  0.039042   
           self_attention                             0.014253   
           self_attention_axial_pe                    0.019065   
           self_attention_fourier_pe                  0.011337   
           transmil                                   0.006650   
           transmil_ourparams                         0.019424   

                                                mean(train/balanced_acc)  \
dataset    model                                                           
camelyon16 abmil                                                0.923064   
           discrete_rel_pos_self_attention                      0.925864   
           distance_aware_self_attention                        0.916758   
           distance_aware_self_attention_fc                     0.977894   
           distance_aware_self_attention_fc_t3                  0.993232   
           just_pool                                            0.639252   
           self_attention                                       0.925499   
           self_attention_axial_pe                              0.886965   
           self_attention_fourier_pe                            0.932341   
           transmil                                             0.996664   
           transmil_ourparams                                   0.959488   

                                                std(train/balanced_acc)  \
dataset    model                                                          
camelyon16 abmil                                               0.003746   
           discrete_rel_pos_self_attention                     0.017280   
           distance_aware_self_attention                       0.014946   
           distance_aware_self_attention_fc                    0.021974   
           distance_aware_self_attention_fc_t3                 0.008117   
           just_pool                                           0.032975   
           self_attention                                      0.016017   
           self_attention_axial_pe                             0.017209   
           self_attention_fourier_pe                           0.010749   
           transmil                                            0.007459   
           transmil_ourparams                                  0.020474   

                                                mean(train/auc)  \
dataset    model                                                  
camelyon16 abmil                                       0.964272   
           discrete_rel_pos_self_attention             0.976428   
           distance_aware_self_attention               0.974490   
           distan